In [17]:
import os
from pytube import YouTube
import subprocess

In [18]:
link = 'https://youtu.be/A8Abf3u0ZIs'

name = 'How to Learn Math for Data Science'

resolution="1080"

path = "a:/youtube"

yt = YouTube(link,
            # use_oauth=True
             )
videos = yt.streams

In [3]:
video = {}
audio = {}
for index, item in (enumerate(videos)):
    item = str(item)
    if resolution in item:
        if 'mp4' in item:
            video['mp4'] = index   
        else:
            video['webm'] = index
    if 'audio' in item:
        if 'mp4' in item:
            item = item.split(' ')
            abr = int(item[3][5:-5])
            if not 'mp4' in audio:
                audio['mp4'] = {
                    'id':index,
                    'abr':abr
                } 
            elif audio['mp4']['abr'] < abr:
                audio['mp4'] = {
                    'id':index,
                    'abr':abr
                }       
        else:
            item = item.split(' ')
            abr = int(item[3][5:-5])
            if not 'webm' in audio:
                audio['webm'] = {
                    'id':index,
                    'abr':abr
                } 
            elif audio['webm']['abr'] < abr:
                audio['webm'] = {
                    'id':index,
                    'abr':abr
                }
if 'mp4' in video:
    vdict = {
        'res':'mp4',
        'video_id': video['mp4'],
        'audio_id': audio['mp4']['id']
        }
else:
    vdict = {
        'res':'webm',
        'video_id': video['webm'],
        'audio_id': audio['webm']['id']
        }

print(vdict)
print(round((videos[vdict['video_id']].filesize + videos[vdict['audio_id']].filesize)/(1024*1024*1024), 2),'GB')

{'res': 'mp4', 'video_id': 5, 'audio_id': 18}
0.21 GB


In [4]:
video_print = videos[vdict['video_id']].download(output_path=path, filename='_video.' + vdict['res'])
print(video_print)
audio_print = videos[vdict['audio_id']].download(output_path=path, filename='_audio.' + vdict['res'])
print(audio_print)

a:/youtube\_video.mp4
a:/youtube\_audio.mp4


In [5]:
args = [
    'ffmpeg',
    '-y',
    '-i',
    os.path.join(path, "_video."+ vdict['res']),
    '-i',
    os.path.join(path, "_audio."  + vdict['res']),
    '-c',
    'copy',
    str(os.path.join(path, name + "." + vdict['res']))
]
subprocess.run(args)


CompletedProcess(args=['ffmpeg', '-y', '-i', 'a:/youtube\\_video.mp4', '-i', 'a:/youtube\\_audio.mp4', '-c', 'copy', 'a:/youtube\\How to Learn Math for Data Science.mp4'], returncode=0)

In [6]:
# Удаляем файлы
os.remove(os.path.join(path, "_video." + vdict['res']))
os.remove(os.path.join(path, "_audio." + vdict['res']))

In [44]:
# Скачиваем субтитры
def get_caption_by_language_name(yt, lang_name : str):
    for caption in yt.caption_tracks:
        if lang_name in caption.name:
            return caption

caption = get_caption_by_language_name(yt, 'English')
# xml_caption = caption.xml_captions
# print(caption.xml_caption_to_srt(xml_caption))

KeyError: 'start'

In [8]:
args = [
    'ffmpeg', # какую прогамму запустить
    '-y', # если файл существует, заменить без уведомления
    '-i', # Указывает что дальше будет файл
    str(os.path.join(path, name + "." + vdict['res'])),
    '-c:v', # указание на кодировку видео
    'h264_nvenc', # кодек видео
    '-crf', # Качесто видео
    '18', # Чем меньше тем выше качество и больше видео
    '-profile:v', # профиль видео для декодера
    'high', # профиль видео
    '-level', # уровень видео для декодера
    '5.2', # уровень видео
    str(os.path.join(path, name +' converted'+".mp4"))
]
# resolution 1440p           
# cpu           speed=1.85x
# cpu+gpu       speed= 1.5x
# gpu(nvenc)    speed=4.29x
subprocess.run(args)
# print(' '.join(args))

ffmpeg -y -i a:\god of war ragnarok 6.webm -crf 18 -profile:v high -level 5.2 a:\god of war ragnarok 6 converted.mp4


In [12]:
# разбитие видео на равные по времени фрагменты
args = [
    'ffmpeg',
    '-i',
    str(os.path.join(path, name +".mp4")),
    '-c',
    'copy',
    '-map',
    '0',
    '-segment_time',
    '02:00:00',
    '-f',
    'segment',
    str(os.path.join(path, name +' output%03d'+".mp4")),
    ]
# subprocess.run(args)
print(' '.join(args))

ffmpeg -i a:/god of war ragnarok the movie.mp4 -c copy -map 0 -segment_time 02:00:00 -f segment a:/god of war ragnarok the movie output\%03d.mp4


Сокращение с использованием длительности 
> ffmpeg -i input.mp4 -ss 00:05:20 -t 00:10:00 -c:v copy -c:a copy output1.mp4
Приведенная выше команда будет принимать входное видео input.mp4, и вырежет из него 10 минут , начиная с 00:05:20

Вырезать, используя определенное время 
> ffmpeg -i input.mp4 -ss 00:05:10 -to 00:15:30 -c:v copy -c:a copy output2.mp4

Вырезать конец видео
> ffmpeg -sseof -600 -i input.mp4 -c copy output5.mp4
> ffmpeg -sseof -00:10:00 -i input.mp4 -c copy output6.mp4

Вырезание с перекодированием
> ffmpeg -ss 00:05:20 -accurate_seek -i input.mp4 -t 00:10:00 -c:v libx264 -c:a aac output7.mp4
> 
В приведенной выше команде мы перекодируем аудио и видео при вырезании видео, а также используем `-accurate_seek` установите флажок, который приведет к более точной длине выходного видео. 
